In [1]:
import collections
from typing import Dict, List, Optional


class Board:
    """Represents a bingo board."""
    
    def __init__(self, rows: List[int]):
        # Assumption: board is square
        assert len(rows) > 0
        assert len(rows) == len(rows[0])
        
        self.rows: List[int] = rows
        self.side_length: int = len(rows)
        
        # Map unmarked numbers to their (row, col) coordinate
        self.coord_map = {}
        for r, row in enumerate(rows):
            for c, num in enumerate(row):
                # Assumption: all numbers on board are unique
                assert num not in self.coord_map
                self.coord_map[num] = (r, c)
        
        # Extra check to make sure I got all the numbers
        assert len(self.coord_map) == self.side_length**2
        
        # Used to determine if the board has won
        self.row_hits: List[int] = [0 for _ in range(len(rows))]
        self.col_hits: List[int] = [0 for _ in range(len(rows[0]))]
            
        self.final_score: int = None
            
    def mark(self, num: int) -> Optional[int]:
        """Returns final score if board has won this turn."""
        if self.final_score:
            # Board has already won in a previous turn
            return
        
        r, c = self.coord_map.pop(num)
        self.row_hits[r] += 1
        self.col_hits[c] += 1
        if self.row_hits[r] == self.side_length or self.col_hits[c] == self.side_length:
            # The board has won!
            self.final_score = num * self.sum_of_unmarked_nums()
            return self.final_score
    
    def sum_of_unmarked_nums(self) -> int:
        return sum(self.coord_map.keys())
        

class Bingo:
    """Represents a bingo game."""
    
    def __init__(self, chosen_numbers: List[int], raw_boards: List[str]):
        # Assume that chosen numbers are all unique
        self.chosen_numbers: List[int] = chosen_numbers
        assert(len(self.chosen_numbers) == len(set(self.chosen_numbers)))
        
        self.boards: List[int] = []
        self.num_to_board_idx: Dict[int, List[int]] = collections.defaultdict(list)
        for board_idx, raw_board in enumerate(raw_boards):
            rows = []
            for raw_row in raw_board.splitlines():
                nums = [int(num) for num in raw_row.split()]
                
                # Track of which boards have which numbers
                for num in nums:
                    self.num_to_board_idx[num].append(board_idx)
                
                rows.append(nums)

            board = Board(rows)
            self.boards.append(board)
        
        # Track final scores in order of wins
        self.final_scores: List[int] = []
        
    def run_turn(self, chosen_num: int):
        boards_to_mark = [self.boards[idx] for idx in self.num_to_board_idx[chosen_num]]
        for board in boards_to_mark:
            maybe_file_score = board.mark(chosen_num)
            if maybe_file_score is not None:
                self.final_scores.append(maybe_file_score)
        
    def run_game(self):
        for chosen_num in chosen_numbers:
            self.run_turn(chosen_num)

In [2]:
input_filename = "input.txt"

with open(input_filename) as input_file:
    chosen_numbers = [int(num) for num in input_file.readline().strip().split(",")]
    raw_boards = input_file.read().strip().split("\n\n")  # I'm lazy!
    
game = Bingo(chosen_numbers, raw_boards)
game.run_game()

# Part 1

In [3]:
print(f"Final score of first winning board:", game.final_scores[0])

Final score of first winning board: 46920


# Part 2

In [4]:
print(f"Final score of last winning board:", game.final_scores[-1])

Final score of last winning board: 12635
